SOP017 - Add app-deploy AD group
================================

Description
-----------

If the Big Data Cluster was installed without an Active Directory group,
you can add one post install using this notebook.

### Steps

### Parameters

In [ ]:
user_or_group_name = "<INSERT USER/GROUP NAME>"
realm = "<INSERT REALM>" # Upper case
sid = "" # To find the SID of the user or the group being added, you can use Get-ADUser or Get-ADGroup PowerShell commands.
role = "appReader"

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Create helper function to run `sqlcmd` against the controller database

In [ ]:
try:
  import pandas
except ModuleNotFoundError:
  !{sys.executable} -m pip install --user pandas
  import pandas
from io import StringIO
pandas.set_option('display.max_colwidth', -1)
name = 'controldb-0'
container = 'mssql-server'

def run_sqlcmd(query):
    command=f"""export SQLCMDPASSWORD=$(cat /var/run/secrets/credentials/mssql-sa-password/password); /opt/mssql-tools/bin/sqlcmd -b -S . -U sa -Q "SET NOCOUNT ON; {query}" -d controller  -s"^" -W > /tmp/out.csv; sed -i 2d /tmp/out.csv; cat /tmp/out.csv"""
    output=stream(api.connect_get_namespaced_pod_exec, name, namespace, command=['/bin/sh', '-c', command], container=container, stderr=True, stdout=True)
    print(output)

print("Function defined")

### Insert user or group into the controller database roles table

In [ ]:
run_sqlcmd(f"INSERT INTO [controller].[auth].[roles] VALUES (N'{user_or_group_name}@{realm}', '{role}')")

### Insert user or group into the controller database active\_directory\_principals tables

In [ ]:
run_sqlcmd(f"INSERT INTO [controller].[auth].[active_directory_principals] VALUES (N'{user_or_group_name}@{realm}', N'{sid}')")

In [ ]:
print("Notebook execution is complete.")